## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. Reemoved pandas definition 'NA' as NaN value,
# which in this case is wrong because in the citipy module, 'NA' = Nambia. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", keep_default_na = False, 
                           na_values=['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 
                                      'N/A', 'n/a', '<NA>', '#NA', 'NULL', 'null', 'NaN', 
                                      '-NaN', 'nan', '-nan', ''])
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.59,26.89,65.34,84,23,12.86,light rain
1,1,Albany,US,42.60,-73.97,48.99,65,56,1.99,broken clouds
2,2,Steamboat Springs,US,40.48,-106.83,30.20,54,1,3.47,clear sky
3,3,Busselton,AU,-33.65,115.33,48.99,94,100,7.20,overcast clouds
4,4,Mokokchung,IN,26.33,94.53,51.06,81,0,2.91,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Mangaratiba,BR,-22.96,-44.04,87.80,52,0,11.41,clear sky
7,7,Atar,MR,20.52,-13.05,70.09,33,100,8.59,overcast clouds
8,8,Sinnamary,GF,5.38,-52.95,83.46,72,99,9.46,heavy intensity rain
9,9,Najran,SA,17.49,44.13,73.40,35,40,1.54,scattered clouds
11,11,San Rafael,AR,-34.62,-68.33,86.52,24,0,9.31,clear sky
14,14,Rikitea,PF,-23.12,-134.97,76.55,81,100,13.98,light rain
15,15,Souillac,MU,-20.52,57.52,75.20,73,40,6.93,scattered clouds
17,17,Risaralda,CO,5.17,-75.77,77.00,69,40,2.24,scattered clouds
28,28,Georgetown,MY,5.41,100.34,77.00,94,20,4.70,few clouds
29,29,Saint-Georges,GD,12.06,-61.75,86.00,74,20,18.34,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

# There are no empty rows (still created new df per isntructions, the previously found NaN was 
# because the country code was labeled as 'NA' where pandas automatically recognized as NaN value, therefore
# in figure #1, removed 'NA' as part of NaN definition list.
clean_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Mangaratiba,BR,87.80,clear sky,-22.96,-44.04,
7,Atar,MR,70.09,overcast clouds,20.52,-13.05,
8,Sinnamary,GF,83.46,heavy intensity rain,5.38,-52.95,
9,Najran,SA,73.40,scattered clouds,17.49,44.13,
11,San Rafael,AR,86.52,clear sky,-34.62,-68.33,
14,Rikitea,PF,76.55,light rain,-23.12,-134.97,
15,Souillac,MU,75.20,scattered clouds,-20.52,57.52,
17,Risaralda,CO,77.00,scattered clouds,5.17,-75.77,
28,Georgetown,MY,77.00,few clouds,5.41,100.34,
29,Saint-Georges,GD,86.00,few clouds,12.06,-61.75,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Above code fills hotel names with white space if cannot be found but cannot readily drop them,
# therefore will fill those cells as NaN.

import numpy as np

hotel_df = hotel_df.replace('',np.nan)
hotel_df.count()

City                   228
Country                228
Max Temp               228
Current Description    228
Lat                    228
Lng                    228
Hotel Name             210
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [11]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Mangaratiba,BR,87.80,clear sky,-22.96,-44.04,Portobello Resort & Safari
7,Atar,MR,70.09,overcast clouds,20.52,-13.05,Oumou Elghoura
8,Sinnamary,GF,83.46,heavy intensity rain,5.38,-52.95,ALETHEIA Appartement Sinnamary
9,Najran,SA,73.40,scattered clouds,17.49,44.13,محمد هادي ال فطيح
11,San Rafael,AR,86.52,clear sky,-34.62,-68.33,Hotel San Rafael
...,...,...,...,...,...,...,...
679,San Marcos,US,71.01,clear sky,33.14,-117.17,Hampton Inn San Marcos
680,Calama,CL,77.00,clear sky,-22.47,-68.93,Park Hotel Calama
681,Thabazimbi,ZA,71.01,clear sky,-24.59,27.41,eagels rest
685,Martapura,ID,78.80,thunderstorm,-3.42,114.85,Montana Hotel Syariah


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]].copy()

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))